In [1]:
from gnn_tracking_hpo.util.paths import add_scripts_path
import optuna
from pathlib import Path
import json
import random
add_scripts_path()

from tune_tcn import suggest_config

In [2]:


study = optuna.create_study()
trial = study.ask()
fixed = suggest_config(trial, ec_project="ec-smaller", ec_hash="2cd19772")

[I 2023-05-18 16:06:51,690] A new study created in memory with name: no-name-41f96456-44d3-4a87-a4bf-ff727211bbf2
[16:06:51] WARNING: Repository /Users/fuchur/Documents/22/git_sync/gnn_tracking/src/gnn_tracking is dirty, commit hash may not be accurate.
[16:06:51] WARNING: Repository /Users/fuchur/Documents/22/git_sync/gnn_tracking_hpo/src/gnn_tracking_hpo is dirty, commit hash may not be accurate.


In [3]:
fixed

{'test': False,
 'gnn_tracking_hash': 'bfa77d1c24f91b4c0b7917ff04896dd4c559c1f4',
 'gnn_tracking_experiments_hash': 'd030a9c9febae84f09a39f9e3aae26adf66c2e7a',
 'train_data_dir': ['/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/graphs_v1/part_1',
  '/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/graphs_v1/part_2',
  '/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/graphs_v1/part_3',
  '/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/graphs_v1/part_4',
  '/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/graphs_v1/part_5',
  '/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/graphs_v1/part_6',
  '/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/graphs_v1/part_7',
  '/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/graphs_v1/part_8'],
 'val_data_dir': '/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/graphs_v1/part_9',
 'n_graphs_train': 247776,
 'n_graphs_val': 200,
 'sector': None,
 'm_mask_orphan_nodes': True,
 'm_use_ec_embedd

In [4]:
outdir = Path("/Users/fuchur/Documents/22/git_sync/gnn_tracking_hpo/config/")
outdir.mkdir(exist_ok=True, parents=True)

In [5]:
grid = {
    "m_L_hc": [3, 6],
    "m_ec_threshold": [0.25, 0.27, 0.29],
    "lw_potential_repulsive": [0.28, 0.32, 0.36],
}

In [20]:
from copy import deepcopy
import itertools


def get_configs(grid, fixed):
    _configs = list(itertools.product(*grid.values()))
    configs = []
    for _c in _configs:
        c = deepcopy(fixed)
        c.update({k: v for k, v in zip(grid.keys(), _c)})
        configs.append(c)
    return configs

In [24]:
configs = get_configs(grid, fixed)
print(len(configs))

18


In [25]:
for i, config in enumerate(configs, start=73):
    with open(outdir / f"config_{i}.json", "w") as f:
        json.dump(config, f, indent=4)